In [1]:
import xml.etree.ElementTree as ET
import numpy as np

def all_xml_to_np(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    if root.tag != "annotation":
        raise Exception("pascal voc xml root element should be annotation, rather than {}".format(root.tag))
    gt_boxes = []
    for elem in root.findall("object/bndbox"):
        startrow = int(elem.find("startrow").text)
        endrow = int(elem.find("endrow").text)
        startcol = int(elem.find("startcol").text)
        endcol = int(elem.find("endcol").text)
        xmin = float(elem.find("xmin").text)
        ymin = float(elem.find("ymin").text)
        xmax = float(elem.find("xmax").text)
        ymax = float(elem.find("ymax").text)

        x1 = float(elem.find("x1").text)
        y1 = float(elem.find("y1").text)
        x2 = float(elem.find("x2").text)
        y2 = float(elem.find("y2").text)
        x3 = float(elem.find("x3").text)
        y3 = float(elem.find("y3").text)
        x4 = float(elem.find("x4").text)
        y4 = float(elem.find("y4").text)

        gt_boxes.append(
            [x1, y1, x2, y2, x3, y3, x4, y4, xmin, ymin, xmax, ymax, startrow, endrow, startcol, endcol]
        )
    np_gt_boxes = np.array(gt_boxes)
    np.set_printoptions(precision=2, suppress=True)
    return np_gt_boxes

In [1]:
import os.path as osp
from sys import argv
import cv2
import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
import subprocess
import random

wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/27_seg/config.py")


set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1, 2]
val = True
# Build dataset

# datasets = build_dataset(cfg.data.train)
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=True, validate=val)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=12.84s)
creating index...
index created!
loading annotations into memory...
Done (t=3.71s)
creating index...
index created!


KeyError: 'LOCAL_RANK'